### Manipulação de Dados com DataFrames no PySpark - UDF

Nesta explicação, vamos detalhar como utilizar **Spark UDF** e **Pandas UDF**, abordando exemplos práticos e suas parametrizações. Veremos como aplicar UDFs com diferentes tipos de entradas e saídas, incluindo estruturas complexas.

---

### Spark UDF (User Defined Function)

As **Spark UDFs** permitem que você crie funções personalizadas em Python e as aplique a colunas de DataFrames no PySpark. As UDFs tradicionais funcionam linha a linha e podem ser aplicadas a uma ou mais colunas.

#### Exemplo 1: UDF com Duas Entradas e Uma Saída (Simples)

Neste exemplo, vamos criar uma UDF para concatenar duas colunas de strings e retornar o resultado.


In [0]:
spark

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Criar um DataFrame de exemplo
data = [("Alice", "Smith"), ("Bob", "Johnson"), ("Charlie", "Brown")]
schema = ["first_name", "last_name"]
df = spark.createDataFrame(data, schema)

# Definir uma função Python para concatenar dois strings
def concat_names(first_name, last_name):
    return first_name + " " + last_name

# Registrar a função como uma UDF
concat_udf = udf(concat_names, StringType())

# Aplicar a UDF ao DataFrame
df_with_full_name = df.withColumn("full_name", concat_udf(df["first_name"], df["last_name"]))

print("DataFrame com Nome Completo:")
display(df_with_full_name)


---

#### Exemplo 2: UDF com Duas Entradas e Uma Saída (Struct)

Neste exemplo, a UDF retorna uma **struct** (estrutura) como saída. Vamos combinar duas colunas e retornar uma estrutura com duas chaves.


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType

# Criar um DataFrame de exemplo
data = [("Alice", "Smith"), ("Bob", "Johnson"), ("Charlie", "Brown")]
schema = ["first_name", "last_name"]
df = spark.createDataFrame(data, schema)

# Definir uma função Python para criar uma estrutura de dados
def create_struct(first_name, last_name):
    return {"first": first_name, "last": last_name}

# Definir o esquema da struct
struct_schema = StructType([
    StructField("first", StringType(), True),
    StructField("last", StringType(), True)
])

# Registrar a função como uma UDF
struct_udf = udf(create_struct, struct_schema)

# Aplicar a UDF ao DataFrame
df_with_struct = df.withColumn("name_struct", struct_udf(df["first_name"], df["last_name"]))

print("DataFrame com Estrutura (Struct):")
display(df_with_struct)


**Explicação do código:**
- A função `create_struct` retorna um dicionário Python, que o Spark converte em uma estrutura (`struct`).
- O esquema da estrutura é definido usando `StructType` com dois campos: `first` e `last`.
- A UDF é aplicada ao DataFrame para gerar a nova coluna `name_struct` com uma estrutura contendo o primeiro e último nome.

---

### Pandas UDF (Vectorized UDF)

As **Pandas UDFs** utilizam a integração do **Apache Arrow** para realizar operações vetorizadas, permitindo que os dados sejam processados em blocos como **Pandas Series** e **DataFrames**, melhorando a eficiência em relação às UDFs tradicionais.

#### Como Parametrizar o Spark para Pandas UDF

Antes de usar Pandas UDFs, é importante ativar o suporte ao Apache Arrow no Spark para garantir a máxima eficiência.


In [0]:
# NAO PRECISA CONFIGURAR AMBIENTE SERVERLESS

# Ativar o uso do Apache Arrow
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")


---

#### Pandas UDF: Series to Series

**Series to Series** é uma UDF onde uma ou mais colunas são passadas como entrada e uma nova coluna (Series) é retornada.

##### Exemplo 1: Pandas UDF com Uma Entrada

Neste exemplo, criamos uma Pandas UDF que converte uma coluna de nomes para maiúsculas.


In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType
import pandas as pd

# Definir uma Pandas UDF para converter texto em maiúsculas
@pandas_udf(StringType())
def to_uppercase(s: pd.Series) -> pd.Series:
    return s.str.upper()

# Aplicar a Pandas UDF ao DataFrame
df_upper = df.withColumn("name_upper", to_uppercase(df["first_name"]))

print("DataFrame com Nome em Maiúsculas:")
display(df_upper)


##### Exemplo 2: Pandas UDF com Duas Entradas

Vamos agora criar uma Pandas UDF que concatena duas colunas (primeiro nome e sobrenome) e retorna o nome completo.


In [0]:
@pandas_udf(StringType())
def concat_names(first_name: pd.Series, last_name: pd.Series) -> pd.Series:
    return first_name + " " + last_name

# Aplicar a Pandas UDF ao DataFrame
df_full_name = df.withColumn("full_name", concat_names(df["first_name"], df["last_name"]))

print("DataFrame com Nome Completo:")
display(df_full_name)


---

#### Pandas UDF: Series to Scalar

**Series to Scalar** é usada para realizar operações agregadas, onde a função recebe uma coluna de dados (Series) e retorna um valor escalar único (por exemplo, média, soma).

**Exemplo:** Calcular a média de uma coluna numérica.


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
import pandas as pd
from pyspark.sql.functions import pandas_udf

# Criar uma massa de dados com valores numéricos
data = [
    (1, 100.0),
    (2, 200.0),
    (3, 300.5),
    (4, 150.25),
    (5, 250.75),
    (6, 325.0),
    (7, 175.4)
]

# Definir o esquema do DataFrame
columns = ["id", "value"]

# Criar o DataFrame
df = spark.createDataFrame(data, columns)

# Mostrar a massa de dados inicial
print("Massa de Dados:")
display(df)

In [0]:

# Definir uma Pandas UDF para calcular a média
@pandas_udf(DoubleType())
def mean_udf(v: pd.Series) -> float:
    return v.mean()

# Aplicar a UDF ao DataFrame para calcular a média
df_mean = df.select(mean_udf(df["value"]).alias("mean_value"))

print("Média dos Valores:")
display(df_mean)


---

#### Pandas UDF: Grouped Map

**Grouped Map** permite que você agrupe os dados por uma ou mais colunas e aplique uma função personalizada a cada grupo. A função recebe um **Pandas DataFrame** para cada grupo e retorna um novo DataFrame.

**Exemplo:** Calcular a soma e a média dos valores dentro de cada grupo.


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import pandas as pd

# Criar uma massa de dados com grupos e valores numéricos
data = [
    ("A", 10),
    ("A", 20),
    ("A", 30),
    ("B", 15),
    ("B", 25),
    ("B", 35),
    ("C", 50),
    ("C", 60)
]

# Definir o esquema do DataFrame
columns = ["group", "value"]

# Criar o DataFrame
df = spark.createDataFrame(data, columns)

# Mostrar a massa de dados inicial
print("Massa de Dados:")
display(df)

In [0]:
# Definir o esquema de saída
schema_out = StructType([
    StructField("group", StringType()),
    StructField("sum_value", IntegerType()),
    StructField("mean_value", DoubleType())
])

# Definir uma função de Pandas que será aplicada por grupo
def group_operations(pdf: pd.DataFrame) -> pd.DataFrame:
    sum_value = pdf["value"].sum()
    mean_value = pdf["value"].mean()
    return pd.DataFrame({
        "group": [pdf["group"].iloc[0]],
        "sum_value": [sum_value],
        "mean_value": [mean_value]
    })

# Aplicar a função com applyInPandas
df_grouped = df.groupBy("group").applyInPandas(group_operations, schema=schema_out)

# Mostrar o resultado
print("Soma e Média por Grupo:")
display(df_grouped)

`[INFO]: Fim Notebook`